In [1]:
import numpy as np
import pandas as pd

# 12.1 Interfacing Between pandas and Model Code

In [5]:
# common workflow:
# - use pandas to load and clean data
# - switch to model library to build the model itself
# important part: feature engineering: data transformations or analytics that extract information from a raw dataset
# switching is easy!
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]
})
data, data.columns

(   x0    x1    y
 0   1  0.01 -1.5
 1   2 -0.01  0.0
 2   3  0.25  3.6
 3   4 -4.10  1.3
 4   5  0.00 -2.0,
 Index(['x0', 'x1', 'y'], dtype='object'))

In [4]:
data.to_numpy()

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [6]:
# to switch back
df2 = pd.DataFrame(data.to_numpy(), columns=["one", "two", "three"])
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [7]:
# if you wish to only use a subset of columns
model_cols = ["x0", "x1"]
data.loc[:, model_cols].to_numpy()

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [8]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'], categories=['a', 'b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [9]:
# pandas has built-in support for one-hot encoding
dummies = pd.get_dummies(data.category, prefix="category", dtype=float)
data_with_dummies = data.drop("category", axis=1).join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1.0,0.0
1,2,-0.01,0.0,0.0,1.0
2,3,0.25,3.6,1.0,0.0
3,4,-4.10,1.3,1.0,0.0
4,5,0.00,-2.0,0.0,1.0


- use `to_numpy()` to easily convert dataframes to numpy arrays
- some libraries do this automatically
- use `df.loc[:, cols]` to select only certain columns
- use `pd.get_dummies()` to create one-hot encoded categorical variables

# 12.2 Creating Model Descriptions with Patsy

`patsy` is a library for describing statistical models with string-based syntax to build model matrices.

In [10]:
data = pd.DataFrame({'x0': [1, 2, 3, 4, 5], 'x1': [0.01, -0.01, 0.25, -4.1, 0.], 'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [11]:
import patsy

# dmatrices is a function that produces design matrices for linear models
y, X = patsy.dmatrices("y ~ x0 + x1", data)

In [12]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [13]:
# these design matrices are essentially numpy arrays with extra information
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [14]:
# the intercept gets added by default
# to remove it:
patsy.dmatrices("y ~ x0 + x1 + 0", data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [18]:
# this design matrix can be passed to e.g. numpy
coef, resid, _, _ = np.linalg.lstsq(X, y, rcond=None)
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

- `patsy` is a simple library for creating statistical models
- use `y, X = patsy.matrices("y ~ x0 + x1", data)` to create a design matrix
- the models are created in string format: `y ~ x0 + x1` means `y` is the dependent variable and `x0` and `x1` are both independent ones
- the returned values are simply numpy arrays with some more metadata
- to exclude an intercept add `+ 0`: `y ~ x0 + x1 + 0`
- the final matrices can be fed into numpy: e.g. `np.linalg.lstsq(X, y, rcond=None)`

## Data Transformations in Patsy Formulas

In [19]:
# you can add python code into patsy formulas
y, X = patsy.dmatrices("y ~ x0 + np.log(np.abs(x1) + 1)", data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [20]:
# patsy also has built-in functions for standardizing (to mean 0 and variance 1) and centering (to mean 0)
y, X = patsy.dmatrices("y ~ standardize(x0) + center(x1)", data)
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [22]:
# sometimes, a model is trained on one dataset but evaluated on another
# when using the above transformations, we have stateful ones, that are based on statistics from the dataset
# we must always use statistics from the original training dataset for these transformations
new_data = pd.DataFrame({
    'x0': [6, 7, 8, 9], 'x1': [3.1, -0.5, 0, 2.3], 'y': [1, 2, 3, 4]
})

# patsy has built-in support to reuse the old values
new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [24]:
# to actually add columns from a dataset, use the I-function
y, X = patsy.dmatrices("y ~ I(x0 + x1)", data)
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

- the model notation of patsy can also include python code: e.g. `y ~ x0 + np.log(np.abs(x1) + 1)`
- patsy will try to find the functions and use it on the data
- there are also built-in functions:
    - `standardize()` can be used to standardize to mean 0 and variance 1
    - `center()` centers the data around the mean
    - `I(x0 + x1)` can be used to add two columns
- when using these stateful transformations, any subsequent data should be transformed with the values of the train data
- for this, patsy has saves the most important information, and it can be reused: `patsy.build_design_matrices([X.design_info], data)`
- other transformations are in the `patsy.builtins` library

## Categorical Data and Patsy

In [26]:
data = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1': [1, 2, 3, 4, 5, 6, 7, 8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})

# nonnumeric terms in a patsy formula are converted to a dummy variable by default
y, X = patsy.dmatrices("v2 ~ key1", data)
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [28]:
# if an intercept is included, one level is left out to avoid collinearity
y, X = patsy.dmatrices("v2 ~ key1 + 0", data)
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [29]:
# numeric columns can also be interpreted as categorical using C()
y, X = patsy.dmatrices("v2 ~ C(key2)", data)
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [30]:
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [31]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [32]:
# we can add interaction terms using key1:key2
y, X = patsy.dmatrices("v2 ~ key1 + key2 + key1:key2", data)
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

- nonnumeric data is transformed to dummy variables by default
- to explicitly interpret a column as categorical, use `C`: `y ~ C(x)`
- interaction terms can be fined like so: `y ~ key1 + key2 + key1:key2`

# 12.3 Introduction to statsmodels

`statsmodels` is a library for performing statistical tests, visualization and data exploration.

## Estimating Linear Models

In [2]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

rng = np.random.default_rng(seed=12345)

def dnorm(mean, variance, size=1):
    """Returns a normally distributed sample of size `size` based on a given mean and variance"""
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * rng.standard_normal(*size)

n = 100
X = np.c_[dnorm(0, 0.4, size=n), dnorm(0, 0.6, size=n), dnorm(0, 0.2, size=n)]
eps = dnorm(0, 0.1, size=n)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps
X

array([[-9.00506021e-01, -1.89429577e-01, -1.02787020e+00],
       [ 7.99252054e-01, -1.54598388e+00, -3.27397080e-01],
       [-5.50654833e-01, -1.20254287e-01,  3.29358994e-01],
       [-1.63915546e-01,  8.24039852e-01,  2.08274848e-01],
       [-4.76512913e-02, -2.13146980e-01, -4.82436357e-02],
       [-4.68576597e-01, -1.43558784e+00, -1.52694953e-01],
       [-8.65068061e-01, -9.63148432e-02,  7.08625055e-01],
       [ 4.10395842e-01,  6.08038650e-01,  1.26222105e-01],
       [ 2.28353201e-01,  1.56467440e-01,  4.06761512e-01],
       [-1.23509905e+00, -3.31585038e-01,  1.76681376e-01],
       [ 1.48463222e+00,  1.43167842e+00, -2.99354280e-01],
       [ 6.12531226e-01,  1.47169718e+00,  6.95582154e-01],
       [-4.80278623e-01, -7.62397041e-02, -5.53712608e-01],
       [ 5.70600290e-01,  6.30092128e-01, -5.34945034e-01],
       [-2.95327118e-01,  3.04024846e-01, -1.91921494e-01],
       [-3.83834219e-02,  6.05303068e-01, -3.26313824e-01],
       [ 4.98908970e-01,  1.12569928e+00

In [3]:
# add intercept
X_model = sm.add_constant(X)
X_model

array([[ 1.00000000e+00, -9.00506021e-01, -1.89429577e-01,
        -1.02787020e+00],
       [ 1.00000000e+00,  7.99252054e-01, -1.54598388e+00,
        -3.27397080e-01],
       [ 1.00000000e+00, -5.50654833e-01, -1.20254287e-01,
         3.29358994e-01],
       [ 1.00000000e+00, -1.63915546e-01,  8.24039852e-01,
         2.08274848e-01],
       [ 1.00000000e+00, -4.76512913e-02, -2.13146980e-01,
        -4.82436357e-02],
       [ 1.00000000e+00, -4.68576597e-01, -1.43558784e+00,
        -1.52694953e-01],
       [ 1.00000000e+00, -8.65068061e-01, -9.63148432e-02,
         7.08625055e-01],
       [ 1.00000000e+00,  4.10395842e-01,  6.08038650e-01,
         1.26222105e-01],
       [ 1.00000000e+00,  2.28353201e-01,  1.56467440e-01,
         4.06761512e-01],
       [ 1.00000000e+00, -1.23509905e+00, -3.31585038e-01,
         1.76681376e-01],
       [ 1.00000000e+00,  1.48463222e+00,  1.43167842e+00,
        -2.99354280e-01],
       [ 1.00000000e+00,  6.12531226e-01,  1.47169718e+00,
      

In [4]:
# fit an ordinary least squares linear regression
model = sm.OLS(y, X)

# must use fit() method first
results = model.fit()
results.params

array([0.06681503, 0.26803235, 0.45052319])

In [5]:
# results has a summary method
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.469
Model:                            OLS   Adj. R-squared (uncentered):              0.452
Method:                 Least Squares   F-statistic:                              28.51
Date:                Wed, 17 Dec 2025   Prob (F-statistic):                    2.66e-13
Time:                        16:54:18   Log-Likelihood:                         -25.611
No. Observations:                 100   AIC:                                      57.22
Df Residuals:                      97   BIC:                                      65.04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [6]:
# dataframes can be passed directly too
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])
data['y'] = y
data[:5]

,col0,col1,col2,y
0,-0.900506,-0.189430,-1.027870,-0.599527
1,0.799252,-1.545984,-0.327397,-0.588454
2,-0.550655,-0.120254,0.329359,0.185634
3,-0.163916,0.824040,0.208275,-0.007477
4,-0.047651,-0.213147,-0.048244,-0.015374


In [7]:
# the dataframe columns can be referenced in string model notation
results = smf.ols("y ~ col0 + col1 + col2", data=data).fit()

# and the parameter names are stored
# the results are stored in pd.Series now, as well!
# note also how we did not include an intercept, but it was created for us
results.params, results.tvalues

(Intercept   -0.020799
 col0         0.065813
 col1         0.268970
 col2         0.449419
 dtype: float64,
 Intercept   -0.652501
 col0         1.219768
 col1         6.312369
 col2         6.567428
 dtype: float64)

In [8]:
# to predict new values
results.predict(data[:5])

0   -0.592959
1   -0.531160
2    0.058636
3    0.283658
4   -0.102947
dtype: float64

- `statsmodels` can be used to estimate linear models
- `sm.OLS(y, X)` is the object to perform an ordinary least squares regression
- use `model.fit()` to perform the estimation
- the resulting object as a method `summary()` to display a summary of the regression
    - and a method `predict()` to perform prediction
- use `sm.add_constant(X)` to add an intercept term (a column of 1) to a matrix of dependent variables
- `statsmodels also supports string notation
    - by doing `smf.ols("y ~ col0 + col1 + col2, data)", this object can be created
    - in this case, the data is a dataframe, and `col0`, `col1`, `col2` and `y` are column identifiers

## Estimating Time Series Processes

In [9]:
init_x = 4
values = [init_x, init_x]
n = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, n)
for i in range(n):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

In [10]:
from statsmodels.tsa.ar_model import AutoReg

max_lags = 5
model = AutoReg(values, max_lags)
results = model.fit()
results.params

array([ 0.02346612,  0.8096828 , -0.42865278, -0.03336517,  0.04267874,
       -0.05671529])

- `AutoReg` can be used to create autoregressive models

# 12.5 Introduction to scikit-learn

In [11]:
%conda install scikit-learn

2 channel Terms of Service accepted
Channels:
 - defaults
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.5.1
    latest version: 25.11.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/nico/miniconda3/envs/data-science

  added / updated specs:
    - scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    joblib-1.5.2               |  py312h06a4308_0         516 KB
    scikit-learn-1.7.2         |  py312h86c3e14_1         9.3 MB
    threadpoolctl-3.5.0        |  py312h7040dfc_1          48 KB
    ------------------------------------------------------------
                                           Total:         9.8 MB

The following NEW packages will be INSTALLED:

  joblib             pkgs/main/linux-64::joblib-1.5.2-py312h

In [12]:
train = pd.read_csv("examples/titanic_train.csv")
test = pd.read_csv("examples/titanic_test.csv")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [18]:
train[train.isna().any(axis=1)].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S


In [19]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [20]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [21]:
# age should be used as a predictor, but it has missing values
# the median will be used here
impute_value = train["Age"].median()
train["Age"] = train["Age"].fillna(impute_value)
test["Age"] = test["Age"].fillna(impute_value)

In [22]:
# a categorical value is added for whether a person was female or not
train["IsFemale"] = (train["Sex"] == "female").astype(int)
test["IsFemale"] = (test["Sex"] == "female").astype(int)

In [23]:
# choose predictors and create numpy arrays
predictors = ["Pclass", "IsFemale", "Age"]

X_train = train[predictors].to_numpy()
X_test = train[predictors].to_numpy()

y_train = train["Survived"].to_numpy()

X_train[:5], y_train[:5]

(array([[ 3.,  0., 22.],
        [ 1.,  1., 38.],
        [ 3.,  1., 26.],
        [ 1.,  1., 35.],
        [ 3.,  0., 35.]]),
 array([0, 1, 1, 1, 0]))

In [24]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [26]:
y_predict = model.predict(X_test)
y_predict[:10] # we do not have true values, so we can not compute the accuracy

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1])

In [27]:
# cross validation
from sklearn.linear_model import LogisticRegressionCV

model_cv = LogisticRegressionCV(Cs=10)
model_cv.fit(X_train, y_train)

,Cs,10
,fit_intercept,True
,cv,None
,dual,False
,penalty,'l2'
,scoring,None
,solver,'lbfgs'
,tol,0.0001
,max_iter,100
,class_weight,None
,n_jobs,None


In [28]:
# or by hand
from sklearn.model_selection import cross_val_score

model = LogisticRegression(C=10)
scores = cross_val_score(model, X_train, y_train, cv=4)
scores

array([0.77578475, 0.79820628, 0.77578475, 0.78828829])

- `scikit-learn` is a popular library for machine learning
- one model is example is `LogisticRegression`
- call `model.fit(x, y)` to fit the model and `model.predict(x)` to predict new values
- `LogisticRegressionCV` has built-in support for cross validation
- alternatively, `cross_val_score(model, x, y, cv)` can be used to